In [2]:
#@title OCR FIX
# Install required dependencies
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract easyocr parsel Pillow numpy pandas opencv-python

import pytesseract
import easyocr
import parsel
import numpy as np
import pandas as pd
import re
import time
import cv2
from PIL import Image
from google.colab import files

def preprocess_barcode_image(image):
    """
    Preprocess image to improve barcode number recognition
    :param image: PIL Image object
    :return: preprocessed PIL Image object
    """
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((1, 3), np.uint8)
    eroded = cv2.erode(binary, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)

    return Image.fromarray(dilated)

def parse_xml(xml):
    """
    parse xml text to be accessed through xpath
    :param xml: xml text
    :type xml: str
    :return: parsel.selector.Selector
    :rtype: object
    """
    parsel.Selector.__str__ = parsel.Selector.extract
    parsel.Selector.__repr__ = parsel.Selector.__str__
    parsel.SelectorList.__repr__ = lambda x: '[{}]'.format(
        '\n '.join("({}) {!r}".format(i, repr(s))
                   for i, s in enumerate(x, start=1))
    ).replace(r'\n', '\n')

    doc = parsel.Selector(text=xml)
    return doc

def clean_barcode_text(text):
    """
    Clean recognized text to extract only numbers
    :param text: recognized text
    :return: cleaned text containing only numbers
    """
    numbers_only = ''.join(filter(str.isdigit, text))
    return numbers_only

def tesseract_read(image):
    """
    extract text from the image using Tesseract
    :param image: PIL Image object
    :return: text
    :rtype: str
    """
    processed_image = preprocess_barcode_image(image)
    custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789'

    text = pytesseract.image_to_string(processed_image, config=custom_config)
    return clean_barcode_text(text)

def easyocr_read(image, reader):
    """
    extract text from the image using EasyOCR
    :param image: PIL Image object
    :param reader: easyocr reader
    :return: text
    :rtype: str
    """
    processed_image = preprocess_barcode_image(image)
    img_array = np.array(processed_image)

    results = reader.readtext(img_array)
    text_results = [x[-2] for x in results]
    text = " ".join(text_results)
    return clean_barcode_text(text)

def compare_ocr(image_path):
    """
    Compare OCR results between Tesseract and EasyOCR for a barcode image
    :param image_path: path to the image
    :return: dict with results
    """
    reader = easyocr.Reader(['en'], gpu=True)
    img = Image.open(image_path)

    tsa_start_time = time.time()
    tsa_result = tesseract_read(img)
    tsa_end_time = time.time()
    tsa_time_used = (tsa_end_time - tsa_start_time)

    easy_start_time = time.time()
    easy_result = easyocr_read(img, reader)
    easy_end_time = time.time()
    easy_time_used = (easy_end_time - easy_start_time)

    return {
        'tesseract_result': tsa_result,
        'tesseract_time': tsa_time_used,
        'easyocr_result': easy_result,
        'easyocr_time': easy_time_used
    }

if __name__ == '__main__':
    print("Upload barcode image:")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\nProcessing barcode image: {filename}")
        results = compare_ocr(filename)

        print("\nResults:")
        print(f"Tesseract result: {results['tesseract_result']}")
        print(f"Tesseract time: {results['tesseract_time']:.2f} seconds")
        print(f"EasyOCR result: {results['easyocr_result']}")
        print(f"EasyOCR time: {results['easyocr_time']:.2f} seconds")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

Saving WhatsApp Image 2024-11-24 at 01.38.32_2c2bbb34.jpg to WhatsApp Image 2024-11-24 at 01.38.32_2c2bbb34.jpg
Saving WhatsApp Image 2024-11-24 at 01.50.47_87eaf36f.jpg to WhatsApp Image 2024-11-24 at 01.50.47_87eaf36f.jpg
Saving WhatsApp Image 2024-11-24 at 01.46.50_49ee5b99.jpg to WhatsApp Image 2024-11-24 at 01.46.50_49ee5b99.jpg

Processing barcode image: WhatsApp Image 2024-11-24 at 01.38.32_2c2bbb34.jpg
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete
Results:
Tesseract result: 
Tesseract time: 0.47 seconds
EasyOCR result: 8796193134
EasyOCR time: 1.30 seconds

Processing barcode image: WhatsApp Image 2024-11-24 at 01.50.47_87eaf36f.jpg

Results:
Tesseract result: 1
Tesseract time: 0.24 seconds
EasyOCR result: 88595331600018
EasyOCR time: 0.43 seconds

Processing barcode image: WhatsApp Image 2024-11-24 at 01.46.50_49ee5b99.jpg

Results:
Tesseract result: 0
Tesseract time: 0.14 seconds
EasyOCR result: 69730431580010
EasyOCR time: 0.10 seconds
